In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'books'

hNeg = True #if true, add negative bigrams for negative reviews
noun = False #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Vocabulário possui 10768 palavras!


In [2]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :];
Ytrain, Yval = Y2[train_index], Y2[test_index];

In [3]:
Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
Xval = Xval[:, index]

#Converte matrizes esparsas para np arrays, para os cálculos da regressão logística
Xtrain = Xtrain.toarray()
Xval = Xval.toarray()

In [4]:
print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
print("----------------------------------------")
print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
print(new_vocabulary)

Número de features antes do chi-quadrado: 10768
----------------------------------------
Número de features após chi-quadrado: 258
[('not', 'happen'), 'aw', ('not', 'wait'), 'gpu', ('not', 'make'), 'wast', 'kagan', 'appli', ('not', 'bad'), ('not', 'buy'), 'info', 'outstand', 'repetit', 'accid', 'repress', 'vagu', 'enjoy', ('not', 'believ'), 'unrealist', 'hiaasen', 'lie', 'eve', 'barber', ('not', 'find'), 'famou', 'secondari', 'bad', 'bare', ('not', 'alway'), ('not', 'not'), 'favorit', 'darl', ('not', 'know'), 'prospect', ('not', 'finish'), ('not', 'want'), 'tri', 'smith', 'alreadi', ('not', 'use'), 'sensit', 'formula', 'terribl', 'say', 'bias', 'weak', 'love', 'intens', 'great', ('never', 'seem'), 'never', 'becki', 'israel', 'help', 'recommend', 'trotski', 'stare', 'think', ('not', 'mention'), ('not', 'thought'), 'pathet', 'script', 'unseen', 'hardli', 'care', 'stick', ('not', 'say'), ('no', 'longer'), 'rambl', 'write', 'lack', 'endless', 'strateg', 'hate', 'wors', 'recycl', 'pay', 'il

In [5]:
def sigmoid(z):
    
    """
    Calcula a funcao sigmoidal  
    """
    
    if isinstance(z, int):
        g = 0
    
    # se z não é um inteiro, significa que é um array e inicia com a dimensão do array
    else:
        g = np.zeros( z.shape );

    g = 1/(1 + np.exp(-z))
  
    return g

In [6]:
def funcaoCustoReg(theta, X, Y, lambda_reg):
    
    """
    Calcula o custo da regressao logística
    """
    
    m = len(Y) #numero de exemplos de treinamento

    J = 0
    grad = np.zeros( len(theta) )
    
    # parâmetro de tolerância para a função sigmoide. 1-htheta não pode ser 
    # menor que eps para evitar erro de precisão numérica
    eps = 1e-15
    
    #print(X)
    
    z = np.dot(X, theta) # z = (theta * X).sum(axis=1)
    
    h_theta = sigmoid(z) 
    cost = -Y * np.log(h_theta+eps) - (1 - Y) * np.log(1 - h_theta + eps)
    J = 1/m * cost.sum()
    
    grad = 1/m *(X.T * (h_theta - Y)).sum(axis=1)
                      
    return J, grad


In [7]:
import scipy
import scipy.optimize 

def treinamento(X, Y, lambda_reg, iteracoes):
    
    # se for vazio, retorna None 
    if len(Y)==0:
        return None
    
    m, n = X.shape # m = qtde de objetos e n = qtde de atributos por objeto
    theta = np.zeros(n) # Inicializa parâmetros que serao ajustados
    # minimiza a funcao de custo
    result = scipy.optimize.minimize(fun=funcaoCustoReg, x0=theta, args=(X, Y, lambda_reg),  
                method='BFGS', jac=True, options={'maxiter': iteracoes, 'disp':False})

    # coleta os thetas retornados pela função de minimização
    theta = result.x
    
    return theta

In [8]:
def predicao(X, theta):
    
    """
    Prediz se a entrada pertence a classe 0 ou 1 usando o parametro
    theta obtido pela regressao logistica
   
    """
    m = X.shape[0]   
#    Ypred = np.zeros(m, dtype="int")
    
#    h = sigmoid( np.dot(X,theta) )
    
#    for i in range(m):
#        if h[i] >= 0.5:
#            Ypred[i] = 1
#        else:
#            Ypred[i] = 0
            
#    acuracia = np.sum(np.array(Ypred[0])==Yval)/len(Yval)
        
#    return Ypred, acuracia
    p = np.zeros(m, dtype=int) 
    z = np.dot(X, theta)
    h_theta = sigmoid(z) 
    
    for i in range(m):
        if h_theta[i] >= 0.5:
            p[i] = 1 
    
    return p

In [ ]:
theta = treinamento(Xtrain, Ytrain, 25, 50)

In [ ]:
Ypred = predicao(Xval, theta)
acuracia = (np.sum(Ypred==Yval)/len(Yval)) * 100
print("Acurácia é "+ str(acuracia))

Acurácia é 92.25


In [ ]:
classes = np.unique(Y)
auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)


	Revocacao   Precisao   F-medida   Classe
	0.912       0.935      0.923      0
	0.933       0.910      0.922      1
	------------------------------------------------
	0.923       0.923      0.923      Média macro
	0.922       0.922      0.922      Média micro

	Acuracia: 0.922


In [ ]:
def curva_aprendizado(Xtrain, Ytrain, Xval, Yval):
   
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    
    perf_train = []
    perf_val = []

    classes = np.unique(Y)
    
    for i in range(10, len(Y)):
        
        theta = treinamento(Xtrain[:i], Ytrain[:i], 25, 50)
        
        Ypred, acuracia = predicao(Xtrain[:i], theta, Ytrain[:i])
        perf_train.append(acuracia)

        Ypred, acuracia = predicao(Xval, theta, Yval)
        perf_val.append(acuracia)

    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()



In [ ]:
#curva_aprendizado(Xtrain, Ytrain, Xval, Yval)

In [ ]:
def gridSearch(X, Y ,Xval, Yval, lambda_reg, iteracoes) :
    bestLambda = lambda_reg[0]
    bestAcc = 0
    for l in lambda_reg :
        theta = treinamento(X,Y, l, iteracoes)
        Ypred = predicao(Xval, theta)
        acuracia = (np.sum(Ypred==Yval)/len(Yval)) * 100
        print("Acurácia é "+ str(acuracia))
        if bestAcc < acuracia :
            bestLambda = l
            bestAcc = acuracia
        
    print("Best Lambda = ", bestLambda)
    print("Acc = ", bestAcc)
    return bestLambda

In [ ]:
lambda_reg = [0.001, 0.01, 0.1, 1, 0.05, 0.5, 5, 10, 50, 100, 500]
l = gridSearch(Xtrain, Ytrain, Xval, Yval, lambda_reg, 100)

Acurácia é 92.0
Acurácia é 92.0
Acurácia é 92.0
Acurácia é 92.0
Acurácia é 92.0
Acurácia é 92.0
Acurácia é 92.0
Acurácia é 92.0
Acurácia é 92.0
Acurácia é 92.0
Acurácia é 92.0
Best Lambda =  0.001
Acc =  92.0


In [ ]:
#K-folds

import k_folds as kf

# separa os dados em k folds
nFolds = 5
folds = kf.stratified_kfolds(Y2, nFolds, classes)

k = 1
resultados=[] # cria uma lista vazia para guardar os resultados obtidos em cada fold
for train_index, test_index in folds:

    print('\n-----------\n%d-fold: \n-----------\n' % (k) )
    
    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    totalFold = len(train_index)+len(test_index)
    
    X_train, X_test = X2[train_index, :], X2[test_index, :];
    Y_train, Y_test = Y2[train_index], Y2[test_index];
    
    X_train, new_vocabulary, index = pp.chi2(X_train, Y_train, vocabulary)
    X_test = X_test[:,index]
    
    # separa os dados de treinamento em treinamento e validacao
    pTrain = 0.8

    ntrain_index, ntest_index = anl.stratified_holdOut(Y_train, pTrain)

    Xtrain, Xval = X_train[ntrain_index, :], X_train[ntest_index, :];
    Ytrain, Yval = Y_train[ntrain_index], Y_train[ntest_index];

        
    #Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
    #Xval = Xval[:, index]

    #Converte matrizes esparsas para np arrays, para os cálculos da regressão logística
    Xtrain = Xtrain.toarray()
    Xval = Xval.toarray()
    
    # chama a função que faz a busca em grade
    lambda_reg = [0.001, 0.01, 0.1, 1, 0.05, 0.5, 5, 10, 50, 100, 500]
    bestRegularization = gridSearch(Xtrain, Ytrain, Xval, Yval,lambda_reg,50)

    # executa o treinamento com o melhor parâmetro de regularização encontrado
    theta = treinamento(Xtrain, Ytrain, bestRegularization, 50)

    X_test = X_test.toarray()
    # classifica os dados de teste
    Y_pred = predicao(X_test, theta)


    # Gera o relatório de desempenho
    print('\n\n\n\t'+"="*50+'\n\tMelhor parametro de regularizacao: %1.6f' %bestRegularization)
    print('\n\tResultado no fold atual usando o melhor parametro encontrado:')
    auxResults = anl.relatorioDesempenho(Y_test, Y_pred,classes, imprimeRelatorio=True)

    # adiciona os resultados do fold atual na lista de resultados
    resultados.append( auxResults ) 
        
    k+=1
    
print('\n-----------\n media-fold: \n-----------\n')
kf.mediaFolds( resultados, classes )


-----------
1-fold: 
-----------

Acurácia é 91.25
Acurácia é 91.25
Acurácia é 91.25
Acurácia é 91.25
Acurácia é 91.25
Acurácia é 91.25
Acurácia é 91.25
Acurácia é 91.25
Acurácia é 91.25
Acurácia é 91.25
Acurácia é 91.25
Best Lambda =  0.001
Acc =  91.25



	Melhor parametro de regularizacao: 0.001000

	Resultado no fold atual usando o melhor parametro encontrado:

	Revocacao   Precisao   F-medida   Classe
	0.865       0.856      0.861      0
	0.855       0.864      0.859      1
	------------------------------------------------
	0.860       0.860      0.860      Média macro
	0.860       0.860      0.860      Média micro

	Acuracia: 0.860

-----------
2-fold: 
-----------

Acurácia é 90.9375
Acurácia é 90.9375
Acurácia é 90.9375
Acurácia é 90.9375
Acurácia é 90.9375
Acurácia é 90.9375
Acurácia é 90.9375
Acurácia é 90.9375
Acurácia é 90.9375
Acurácia é 90.9375
Acurácia é 90.9375
Best Lambda =  0.001
Acc =  90.9375



	Melhor parametro de regularizacao: 0.001000

	Resultado no fold atual